<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/GEMINI3_LeJEPA_MEDICAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
import re
import torch
from google.colab import userdata
from datetime import datetime
import pytz

# --- GEMINI CLIENT SETUP ---
import google.generativeai as genai
from google.generativeai import types

try:
    # 1. Retrieve the API key securely from Colab's user data
    GOOGLE_API_KEY = userdata.get('GEMINI')
    if not GOOGLE_API_KEY:
        raise ValueError("Error: 'GEMINI' API key not found in Colab Secrets.")

    genai.configure(api_key=GOOGLE_API_KEY)

    # 2. Define the target model ID (The core reasoning engine)
    GEMINI_MODEL_ID = 'gemini-3-pro-preview'
    gemini_model = genai.GenerativeModel(GEMINI_MODEL_ID)
    print(f"Gemini client successfully configured for model: {GEMINI_MODEL_ID}.")

    # --- Utility Function: Load Image for Gemini API ---
    # FIX: Uses genai.upload_file() to handle local file upload reliably.
    def load_image_part(file_path):
        """Loads and uploads the local file, returning the File object."""
        if not os.path.exists(file_path):
             raise FileNotFoundError(f"Image file not found at {file_path}")

        # Use the genai.upload_file utility which is the recommended way
        # to handle local files and ensures compatibility.
        uploaded_file = genai.upload_file(file_path)
        print(f"File uploaded to Gemini API: {uploaded_file.name}")
        return uploaded_file

except Exception as e:
    print(f"FATAL ERROR during Gemini setup: {e}")

    # Define a Mock client as a fallback for the main loop
    class MockGeminiClient:
        def __init__(self):
            self.iteration = 0
            self.is_mock = True
        def generate_content(self, contents, **kwargs):
            self.iteration += 1
            # Mock content simulates the Gemini model's output refinement
            if self.iteration == 1:
                 # Simulates the successful, constraint-adherent output
                 mock_content = f"### RADIOLOGICAL CLINICAL REPORT\n\n1. Perception/Fact Extraction: Colonic distention, mural thickening, and pericolonic fat stranding were observed.\n\n2. Diagnosis & Plan: Most Likely Diagnosis: Stercoral Colitis. Acute Management: Endoscopic removal (flexible sigmoidoscopy). Long-Term: Pelvic-Floor Physical Therapy, Anorectal Manometry, and Biofeedback. Potential Complications: Focal-Pressure Necrosis and Perforation."
            else:
                 # Should not happen in a successful mock run
                 mock_content = "Mock client completed all expected outputs."

            return type('MockResponse', (object,), {'text': mock_content})()

    gemini_model = MockGeminiClient()
    GEMINI_MODEL_ID = "MOCK_GEMINI_3_PRO"
    print("Using Mock Gemini Client for execution.")

# --- 2. Agent Class Definitions ---

class ImageAnalysisAgent:
    """The Gemini 3 Pro Reasoning Engine, handling both perception and diagnosis."""
    def __init__(self, model_instance):
        self.model = model_instance

    def analyze_image_and_diagnose(self, prompt, image_part):
        """Sends the prompt and image directly to Gemini for a combined output."""
        if hasattr(self.model, 'is_mock'):
             return self.model.generate_content(contents=prompt).text

        # The core multimodal API call accepts the File object
        response = self.model.generate_content(
            contents=[prompt, image_part]
        )
        return response.text.strip()


class ValidationAgent:
    """Pillars 3 & 4: Constraint and Safety Enforcement"""
    def __init__(self):
        self.expected_patterns = {
            "diagnosis": [r"stercoral\s+colitis"],
            "acute_procedure": [r"endoscopic\s+(?:removal|disimpaction|evacuation|sigmoidoscopy)"],
            "long_term_dx": [r"anorectal\s+manometry", r"puborectalis\s+dysfunction"],
            "long_term_tx": [r"pelvic-floor\s+physical\s+therapy", r"biofeedback"],
            "complications": [r"perforation", r"necrosis", r"ischemia"]
        }

    def validate_output(self, model_output):
        issues = []
        lower_output = model_output.lower()

        if not any(re.search(pattern, lower_output) for pattern in self.expected_patterns["diagnosis"]):
            issues.append("Diagnosis is missing the precise medical term 'Stercoral Colitis'.")
        if not any(re.search(pattern, lower_output) for pattern in self.expected_patterns["acute_procedure"]):
            issues.append("The acute intervention plan lacks specific mention of endoscopic methods for evacuation.")
        if not any(re.search(pattern, lower_output) for pattern in self.expected_patterns["long_term_dx"]):
            issues.append("Long-term management is incomplete; assessment (e.g., anorectal manometry) is missing.")
        if not any(re.search(pattern, lower_output) for pattern in self.expected_patterns["long_term_tx"]):
            issues.append("Long-term therapy is incomplete; must explicitly include 'pelvic-floor physical therapy' or 'biofeedback.'")
        if not any(re.search(pattern, lower_output) for pattern in self.expected_patterns["complications"]):
            issues.append("Potential complications such as necrosis or perforation are not explicitly noted.")

        return issues if issues else ["Output aligns with expected clinical criteria."]

class PromptEngineerAgent:
    """Pillar 5: Adaptive Steering and Workflow Management"""
    def __init__(self):
        self.base_prompt = (
            "You are a specialist radiologist. Analyze the provided multi-slice CT scan images. "
            "Patient: 23-year-old male with autism and chronic constipation, presenting with acute abdominal pain. "
            "You MUST perform two tasks:\n"
            "1. **Perception/Fact Extraction:** First, explicitly list the raw, verifiable radiological signs (e.g., colonic distention, mural thickening, fat stranding) you observe.\n"
            "2. **Diagnosis & Plan:** Based on these facts and the patient history, provide the **most likely diagnosis** and a detailed management plan. Your plan MUST adhere to the following clinical constraints for a high-risk case:\n"
            "   - Explicitly use the term **Stercoral Colitis** if the findings support it.\n"
            "   - **Acute Management** MUST include **endoscopic removal** (e.g., sigmoidoscopy).\n"
            "   - Explicitly note the risk of **perforation or necrosis**.\n"
            "   - **Long-Term Management** MUST include assessment (e.g., anorectal manometry) and therapy (e.g., pelvic-floor physical therapy or biofeedback).\n"
            "Provide your final output in a clean clinical report format."
        )
        self.iteration = 0

    def refine_prompt(self, validation_feedback):
        self.iteration += 1
        refined_prompt = self.base_prompt

        if validation_feedback and "Output aligns" not in validation_feedback[0]:
            refinement_instruction = f" **CRITICAL REFINEMENT (Iteration {self.iteration}):** The previous response failed to meet specific clinical requirements. You MUST now ensure the corrected output adheres to **ALL** the original constraints. Specifically, address the following deficiencies: {', '.join(validation_feedback)}. Re-evaluate your previous analysis and integrate these missing elements."
            refined_prompt += refinement_instruction

        return refined_prompt

# --- 3. Main Execution Loop (The Orchestrator) ---

def run_multi_agent_experiment_gemini():
    print(f"\n--- Gemini 3 Pro Agent Experiment Started at {datetime.now(pytz.timezone('EST')).strftime('%I:%M %p %Z on %B %d, %Y')} ---")

    # Initialize agents
    prompt_engineer = PromptEngineerAgent()
    image_analyzer = ImageAnalysisAgent(gemini_model)
    validator = ValidationAgent()

    # NOTE: This path MUST be valid for live calls
    image_path = "/content/drive/MyDrive/datasetsmedical/image.png"

    try:
        if not hasattr(gemini_model, 'is_mock'):
            image_part = load_image_part(image_path)
        else:
            image_part = None
            print("Running in Mock Mode. Image part ignored.")
    except Exception as e:
        print(f"FATAL ERROR: Could not load image file from Drive. {e}")
        return


    # Step 1: Initialize Prompt and Loop
    max_iterations = 5
    current_iteration = 0
    validation_result = []

    refined_prompt = prompt_engineer.refine_prompt(validation_result)

    # --- ENHANCED LOGGING (Displays the full power of the Prompt Engineer) ---
    print("\n--- PROMPT ENGINEER AGENT: Initial Steering Command ---")
    print(f"Goal: Achieve a single-shot clinically accurate diagnosis and plan.")
    print(f"Model: {GEMINI_MODEL_ID}")
    print(f"--- Full Initial Prompt ---")
    print(refined_prompt)
    print("---------------------------------------------------------")
    # -------------------------------------------------------------------------

    while current_iteration < max_iterations:
        print(f"\n--- Iteration {current_iteration + 1} ({GEMINI_MODEL_ID}) ---")

        # Step 2: Call the single Gemini model for analysis and diagnosis
        try:
            model_output = image_analyzer.analyze_image_and_diagnose(refined_prompt, image_part)
        except Exception as e:
            print(f"\n[EXCEPTION CAUGHT] Error during Gemini inference: {e}")
            break


        print(f"Gemini Analysis Result: {model_output[:80].replace('\\n', ' ') + '...'}")

        # Step 3: Validate Output (Constraint Check)
        validation_result = validator.validate_output(model_output)
        print(f"Validation Agent Feedback: {validation_result}")

        # Step 4: Check for Convergence
        if "Output aligns" in validation_result[0]:
            print("\n**AGENTS CONVERGED: Validation criteria met. Stopping iteration.**")
            break

        # If validation fails, prepare the prompt for the next loop
        refined_prompt = prompt_engineer.refine_prompt(validation_result)
        current_iteration += 1

    if current_iteration == max_iterations and (not model_output or "Output aligns" not in validation_result[0]):
        print("\nMaximum iterations reached without convergence.")

# Execute the experiment
if __name__ == "__main__":
    run_multi_agent_experiment_gemini()

Gemini client successfully configured for model: gemini-3-pro-preview.

--- Gemini 3 Pro Agent Experiment Started at 11:56 PM EST on November 22, 2025 ---
File uploaded to Gemini API: files/ul54hjvy8chv

--- PROMPT ENGINEER AGENT: Initial Steering Command ---
Goal: Achieve a single-shot clinically accurate diagnosis and plan.
Model: gemini-3-pro-preview
--- Full Initial Prompt ---
You are a specialist radiologist. Analyze the provided multi-slice CT scan images. Patient: 23-year-old male with autism and chronic constipation, presenting with acute abdominal pain. You MUST perform two tasks:
1. **Perception/Fact Extraction:** First, explicitly list the raw, verifiable radiological signs (e.g., colonic distention, mural thickening, fat stranding) you observe.
2. **Diagnosis & Plan:** Based on these facts and the patient history, provide the **most likely diagnosis** and a detailed management plan. Your plan MUST adhere to the following clinical constraints for a high-risk case:
   - Expli